<center><img src="WHU.png" alt="武汉大学校徽"/></center>
<center><font size="7">实验报告</font></center>

# 神经网络可视化实验
专业: 临床医学(八年制)  
班级: 3班  
姓名: 闻人星湘  
指导教师: 王翀  

## 实验环境说明
本地实验的配置信息如下:
1. 该实验所使用的操作系统为 Ubuntu 24.04 LTS, CPU 为 AMD RYZEN 5000 Series 7, GPU 为 Geforce RTX 3050.
2. 该实验使用的 Python 版本为 3.12.3, CUDA Toolkit 版本为 12.4, cuDNN 版本为 9.8, [Pytorch](https://pytorch.org/) 版本为 2.6.

云端实验的配置信息如下:  
&emsp;&emsp;该实验所使用的浏览器为 Firefox, 浏览器中使用 [TensorFlow Playground](https://playground.tensorflow.org/) 进行探究.

In [3]:
# 导入需要的第三方库
import torch
import seaborn as sns
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.datasets import make_circles

# 根据实际情况选择训练硬件
device = torch.accelerator.current_accelerator().type \
    if torch.accelerator.is_available() else "cpu"
print(f"使用{device}训练模型.")

使用cuda训练模型.



## 实验目标
1. 理解神经网络的基本结构: 了解神经网络的层(Layer), 包括输入层(Iuput Layer), 隐藏层(Hidden Layer), 输出层(Output Layer); 了解神经网络的激活函数(Activation Function), 包括但不限于线性整流函数(ReLU), 双曲正弦函数(Tanh), S型函数(Sigmoid), 归一化指数函数(Softmax)等. 学会合理配置神经网络层数以及每层神经元数.
2. 观察神经网络的训练过程: 了解损失函数(Loss Function), 参数更新策略(Parameter Update Strategy), 学习率(Learning Rate), 前向传播(Forward), 反向传播(Backward), 批大小(Batch Size), 训练轮数(Epoch)的概念, 学会通过损失曲线(Loss Curve)判断网络是否在学习.
3. 掌握神经网络的超参数调优: 学会调整 Learning rate, 理解它对训练速度的影响; 尝试不同激活函数, 观察分类效果的变化; 了解正则化(Regularization)以及如何防止模型过拟合.
4. 理解神经网络能解决的问题类型: 通过不同数据集(如圆形分布, 螺旋分布等)，理解神经网络如何分类复杂数据, 学会通过添加特征(如$X^2$​, $\sin(x)$等​)提升模型性能.
5. 培养动手实践和问题解决能力: 通过调整参数和结构，完成分类任务挑战; 学会分析实验结果，提出改进方案.

## 实验具体内容

### 初识神经网络
实验设置: 使用圆形分布数据集, 设置学习率为 0.03, 激活函数为 Tanh, 隐藏层为 1 层 4 个神经元.  
实验结果如下:
<img src="First.png" alt="初识神经网络">

&emsp;&emsp;可以观察到, 背景颜色原本为无色(还未开始训练), 而后迅速变为橙色包裹蓝色, 蓝色朝某个方向开口, 随着训练时间增加, 蓝色迅速闭合. 分类前后明显可以在背景颜色和数据点(Sample)的位置关系看到差异: 训练结束后数据点和背景对应得很好.  
&emsp;&emsp;损失曲线均呈S型, 初期和末期下降速度慢, 中期下降较快. 训练结束后训练数据上的损失和测试数据上的损失差异很小(0.01)且均较低, 表明模型有良好的泛化能力, 但是数据集和模型非常简单(没有噪声的引入, 隐藏层数量和神经元数量小). 该模型并不具有实际应用价值.
&emsp;&emsp;分类效果图和损失曲线反映了模型学习的实时状态, 两者在反映模型学习情况上相互补充, 我们可以根据两者及时地调整模型.  
&emsp;&emsp;若损失没有降到 0.01 以下, 说明模型拟合能力不足(在该次实验中几乎不会出现), 从数据集和模型本身来看, 有以下几点原因:
1. 模型过于简单
2. 特征工程不足
3. 训练时间不足
4. 数据量不足
5. 正则化过度
6. 学习率设置不当
7. 数据质量问题
8. 优化算法选择不当

&emsp;&emsp;如果该模型真的出现了这个问题, 最可能的是由于深度模型只能得到局部最优解, 而不是全局最优解, 随机参数初始化过程导致了这个问题.  
&emsp;&emsp;以下复现了网站上的代码, 其中达到要求(损失降到 0.01 以下)实测共训练约 330 轮, 约为网站实测值(846轮)的 $\frac{1}{2}$:


In [4]:
# 随机生成数据, 内圆和外圆各 200 个点
X, y = make_circles(400)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# 定义用于该问题的数据集类
class CircleDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return X[index], y[index]

# 初始化数据集
circle_dataset = CircleDataset(X, y)
train_dataset, test_dataset = random_split(circle_dataset, [200, 200])
# 初始化数据加载器
train_loader = DataLoader(train_dataset, batch_size=10)
test_loader = DataLoader(test_dataset, batch_size=10)

In [5]:
# 构建最简单的模型
model = nn.Sequential(
    nn.Linear(2, 4), nn.Tanh(),
    nn.Linear(4, 4), nn.Tanh(),
    nn.Linear(4, 2)
).to(device)
# 定义损失函数为交叉熵损失
loss_fn = nn.CrossEntropyLoss()
# 定义优化器为随机梯度下降
optimizer = torch.optim.SGD(model.parameters(), lr=0.03)

In [6]:
def train_process(dataloader, model, loss_fn, optimizer):
    # 设置模型为训练模式
    model.train()
    # 初始化训练损失
    train_loss = 0
    # 遍历数据集
    for batch, (X, y) in enumerate(dataloader):
        # 将数据移动到训练硬件上
        X, y = X.to(device), y.to(device)
        # 使用模型获取预测值并计算损失
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()
        # 反向传播
        loss.backward()  # 计算梯度
        optimizer.step()  # 更新参数
        optimizer.zero_grad()  # 清空梯度
    # 返回该次的平均损失
    return train_loss / len(dataloader)

def test_process(dataloader, model, loss_fn):
    # 设置模型为评估模式
    model.eval()
    # 初始化测试损失和正确数
    test_loss, correct = 0, 0
    # 不计算梯度
    with torch.no_grad():
        # 遍历数据集
        for X, y in dataloader:
            # 将数据移动到训练硬件上
            X, y = X.to(device), y.to(device)
            # 使用模型获取预测值并计算损失
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            # 计算该批次中的正确数量并累加到 correct 中
            correct += (pred.argmax(1) == y) \
                .type(torch.float).sum().item()
    return test_loss / len(dataloader), \
        correct / len(dataloader.dataset)

def train(
    epochs, train_loader, test_loader, model,
    loss_fn, optimizer, shutdown=False, print_flag=True
):
    for t in range(epochs):
        train_loss = train_process(
            train_loader, model, loss_fn, optimizer
        )
        test_loss, test_acc = test_process(test_loader, model, loss_fn)
        if (t % 10 == 0) and print_flag:
            print(
                f"epoch {t}, train loss: {train_loss:.4f}, "
                f"test loss: {test_loss:.4f}, test acc: {test_acc:.4f}"
            )
        if shutdown and test_loss < 0.1:
            return train_loss, test_loss, t + 1
    return train_loss, test_loss, epochs

# 以下为运行该次试验的代码, 需要运行时请取消注释
# train(350, train_loader, test_loader, model, loss_fn, optimizer)

### 探索学习率对训练速度的影响
| 实验序号 | 学习率 | 测试损失达到 0.01 时的 epoch 数 |
| :-: | :-: | :-: |
| 1 | 0.003 | 约 5500 |
| 2 | 0.03 | 846 |
| 3 | 0.1 | 127 |
| 4 | 1 | (模型无法稳定) |

&emsp;&emsp;从表中可以看出, 学习率与训练所需轮数大致成反比, 但学习率过高会导致模型无法收敛: 损失曲线一直在波动, 无法平稳, 分类效果比较差.  
&emsp;&emsp;这说明模型的学习率设置不合理或者 Batch 大小不合理, 不能很好地匹配给定的学习率.  
&emsp;&emsp;然而学习率不能太小, 这会导致模型需要很多轮才能收敛, 进而去拟合数据.  
&emsp;&emsp;以下是寻找最好学习率的代码, 若到 1500 轮测试损失都不能小于 0.01, 则模型被认为不能及时收敛, 仅探讨学习率区间 0.01-0.20, 运行结果为 0.11-0.17 为比较好的学习率:

In [ ]:
def detect_lr():
    for i in range(20):
        lr = (i+1)*0.01
        
        loss_fn = nn.CrossEntropyLoss()
        model = nn.Sequential(
            nn.Linear(2, 4), nn.Tanh(),
            nn.Linear(4, 4), nn.Tanh(),
            nn.Linear(4, 2)
        ).to(device)
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
        
        train_loss, test_loss, epochs = train(
            1500, train_loader, test_loader,
            model, loss_fn, optimizer, True, False
        )
        print(f"lr: {lr}, epochs: {epochs}.")

# 以下为运行该次试验的代码, 需要运行时请取消注释
# detect_lr()

## 参考文献
[动手学深度学习(第二版)](动手学深度学习(第二版).pdf)